In [2]:
! python3 --version
! pip3 --version
! pip3 install mysql-connector-python

Python 3.7.15
pip 22.3.1 from /home/glue_user/.local/lib/python3.7/site-packages/pip (python 3.7)
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
! cat /etc/os-release
# ! cp ./mysql-connector-java-5.1.49.jar $SPARK_HOME/jars
# ! ls $SPARK_HOME/jars

NAME="Amazon Linux"
VERSION="2"
ID="amzn"
ID_LIKE="centos rhel fedora"
VERSION_ID="2"
PRETTY_NAME="Amazon Linux 2"
ANSI_COLOR="0;33"
CPE_NAME="cpe:2.3:o:amazon:amazon_linux:2"
HOME_URL="https://amazonlinux.com/"


In [102]:
import sys
import boto3
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, lit, concat, create_map, when
from pyspark.sql.types import StringType, DoubleType
import pyspark.sql.functions as f
from itertools import chain
from awsglue.context import GlueContext
from awsglue.job import Job

import csv, io
import mysql.connector
from functools import partial
from itertools import chain

In [5]:
spark = SparkSession.builder \
           .appName('123.com') \
           .config("spark.jars", "mysql-connector-java-5.1.49.jar") \
           .getOrCreate()

In [6]:
class DB_Connection():
    def __init__(self, db_name, hostname = None, user = None, password = None, driver_type = "mysql", driver_port = 3306, glue_cilent = None, connection_name = None):
        try:
            self.glue = False
            if glue_cilent is not None and connection_name is not None:
                self.glue = True
                connection = glue_cilent.get_connection(Name=connection_name)
                self.hostname = connection['Connection']['ConnectionProperties']['JDBC_CONNECTION_URL']
                self.hostname = self.hostname.split('/')[2].split(':')[0]
                self.user = connection['Connection']['ConnectionProperties']['USERNAME']
                self.password = connection['Connection']['ConnectionProperties']['PASSWORD']
            else:
                self.hostname = hostname
                self.user = user
                self.password = password
            
            self.conn = None
            self.db_name = db_name
            self.driver_type = driver_type
            self.driver_port = driver_port
            
            self.jdbc_url = f'jdbc:{self.driver_type}://{self.hostname}:{self.driver_port}/{self.db_name}'
        except Exception as e:
            print(f"Error getting connection settings for DB: {db_name} at {hostname}\n{e}")
    def get_options(self, table):
        return {'url': self.jdbc_url,'user': self.user, 'password': self.password, 'dbtable': table}
    def get_conn(self, dbType = "mysql", mandatory = False, ssl_disabled= True):
        # self.conn.is_connected()
        if dbType == "mysql":
            self.conn =  mysql.connector.connect(user=self.user, password=self.password, host=self.hostname, database=self.db_name, ssl_disabled= ssl_disabled)
        if mandatory: assert self.conn is not None, f"Did not manage to connect to {dbType}"
        return self.conn
    def close_conn(self):
        if self.conn is not None and self.conn.is_connected():
            self.conn.close()

In [568]:
# IMPORTANT VARIABLES
TENANT = 'scis'
conversionRateMapping = {
    "USD": 1.35,
    "SGD": 1
}
csvFile = './testSpend.csv'
queueName = 'CampaignToCard'

In [508]:
#  Get relevant data

s3_df_ = spark.read.format("csv").option("header", "true").load(csvFile)
s3_df_ = s3_df_.withColumn("remark", lit(None).cast(StringType()))


In [509]:
# Get relevant connections
campaign_db2_c = DB_Connection(db_name = 'campaign_db', hostname = 'host.docker.internal', driver_type = "mysql", driver_port = 8003, user = 'admin', password = 'adminadmin')
print(campaign_db2_c.get_options("campaign"))

{'url': 'jdbc:mysql://host.docker.internal:8003/campaign_db', 'user': 'admin', 'password': 'adminadmin', 'dbtable': 'campaign'}


In [511]:
print(s3_df.count())
print(s3_df.columns)
s3_df.limit(5).show()

5
['id', 'transaction_id', 'merchant', 'mcc', 'currency', 'amount', 'transaction_date', 'card_id', 'card_type', 'remark']
+--------------------+--------------------+--------------------+----+--------+-------+----------------+--------------------+-------------+------+
|                  id|      transaction_id|            merchant| mcc|currency| amount|transaction_date|             card_id|    card_type|remark|
+--------------------+--------------------+--------------------+----+--------+-------+----------------+--------------------+-------------+------+
|055cd9e8-e9f4-4f8...|5cbd48a13727468fe...|           Bayer LLC|1047|     SGD| 158.54|        3/9/2021|605ec742-0ac4-43e...|scis_shopping|  null|
|4a25d76c-2db7-47b...|a750165d557443feb...|Transaction Recor...|   0|     SGD|1252.95|        3/9/2021|605ec742-0ac4-43e...|scis_shopping|  null|
|43dd9d38-9b7f-4a2...|41a928d858e21fd36...|     Bahringer Group|8462|     SGD|  37.72|        3/9/2021|2379df1a-d1fe-415...|scis_shopping|  null|
|5

In [539]:
s3_df = s3_df_

In [540]:
# Check MCC
errorDf = s3_df.filter(col('mcc').isNull())
s3_df = s3_df.filter(col('mcc').isNotNull())

errorDf = errorDf.na.fill(value="MCC is empty!",subset=["remark"])

print(errorDf.count())
print(s3_df.count())

0
5


In [541]:
# Check duplicates rows
s3_df = s3_df.join(
    s3_df.groupBy('transaction_id').agg((f.count("*")>1).cast("int").alias("Duplicate_indicator")),
    on='transaction_id',
    how="inner"
)
errorDf = errorDf.union(s3_df.filter(col("Duplicate_indicator") == 1).select(errorDf.columns))
s3_df = s3_df.filter(col("Duplicate_indicator") != 1).select(errorDf.columns)
errorDf = errorDf.na.fill(value="Duplicate rows",subset=["remark"])
print(errorDf.count())
print(s3_df.count())

0
5


In [542]:
# Check card type
cardTypesDf = spark.read.format("jdbc").option('driver', 'com.mysql.jdbc.Driver').option("url", campaign_db2_c.jdbc_url).option("user", campaign_db2_c.user).option("password", campaign_db2_c.password).option("dbtable", "card_type").load()
cardTypesDf = cardTypesDf.filter(col("tenant") == TENANT)
cardTypesDf = cardTypesDf.withColumnRenamed("id", "cardTypeId")

# interest cols
colsInterest = s3_df.columns
colsInterest.append("cardTypeId")
s3_df = s3_df.join(cardTypesDf, s3_df.card_type ==  cardTypesDf.name,"left").select(colsInterest)

# Check card type
errorDf = errorDf.union(s3_df.filter(col('cardTypeId').isNull()).select(errorDf.columns) )
s3_df = s3_df.filter(col('cardTypeId').isNotNull())

errorDf = errorDf.na.fill(value="Card type not registered in DB!",subset=["remark"])
print(errorDf.count())
print(s3_df.count())

0
5


In [543]:
s3_df = s3_df.withColumn("rewardAmountBase", lit(0).cast(DoubleType()) ).withColumn("rewardAmountCat", lit(0).cast(DoubleType()) ).withColumn("rewardAmountCampaign", lit(0).cast(DoubleType()) )
s3_df.limit(5).show()

+--------------------+--------------------+--------------------+----+--------+-------+----------------+--------------------+-------------+------+----------+----------------+---------------+--------------------+
|                  id|      transaction_id|            merchant| mcc|currency| amount|transaction_date|             card_id|    card_type|remark|cardTypeId|rewardAmountBase|rewardAmountCat|rewardAmountCampaign|
+--------------------+--------------------+--------------------+----+--------+-------+----------------+--------------------+-------------+------+----------+----------------+---------------+--------------------+
|20285a85-788f-47e...|3cc0c41872cc33b39...|               Smith|8173|     USD| 580.94|        3/9/2021|2379df1a-d1fe-415...|scis_shopping|  null|         4|             0.0|            0.0|                 0.0|
|5b2d5c02-75fc-404...|8a46f127d7e3ec1e5...|       Langworth Inc|9997|     SGD|  64.77|        3/9/2021|2379df1a-d1fe-415...|scis_shopping|  null|         4|

In [471]:
customCatDf.loc[( customCatDf.apply(lambda x: x['name'].upper() in 'AXS Payments'.upper(), axis=1) ) & (customCatDf['mcc'] == int(9399)), 'merchant'].tolist

<bound method IndexOpsMixin.tolist of Series([], Name: merchant, dtype: object)>

In [544]:
def processingFunc(partitionData, conversionRateMapping, mccExclusionDf, campaignRulesDf, customCatDf):
#     print("Start")
    remarks_mapping = {
        "default": ["Base reward", "Category reward", "Campaign reward"],
        "cols": ["rewardAmountBase","rewardAmountCat" , "rewardAmountCampaign"],
        "rate": ['cashback', 'miles', 'miles', 'points']
    }
    for row in partitionData:
        row = row.asDict()
        row['mcc'] = int(row['mcc'])
        row['cardTypeId'] = int(row['cardTypeId'])
        row['remark'] = ''
        try:
            check_exclusion = len(mccExclusionDf.loc[(mccExclusionDf['card_type_id'] == row['cardTypeId']) & (mccExclusionDf['mcc_mcc'] == row['mcc'])]) > 0
            currencyRate = conversionRateMapping.get(row['currency'].upper(), False)
            if check_exclusion:
                print( f"{row['mcc']} is part of the exclusion list!")
                row['remark'] = f"No reward due to exclusion - {row['mcc']}"
            elif not currencyRate:
                print(f"{row[currency]} is not in currency mapping!")
                row['remark'] = f"{row[currency]} is not in currency mapping!"
                row['rewardAmountBase'] = None
            else:
                amount = float(row['amount']) * currencyRate
                isForeign = row['currency'] != "SGD"
                
                # Calculate base, category and campaigns
                conditions_base = campaignRulesDf.loc[(campaignRulesDf['title'] == 'base') & (campaignRulesDf['card_type_id'] == row['cardTypeId'])]
                conditions_category = campaignRulesDf.loc[(campaignRulesDf['title'] == 'category') & (campaignRulesDf['card_type_id'] == row['cardTypeId'])]
                conditions_campaigns = campaignRulesDf.loc[(campaignRulesDf['end_date'].notna()) & (campaignRulesDf['card_type_id'] == row['cardTypeId'])]
                conditions_arr = [conditions_base, conditions_category, conditions_campaigns]
                
                for i in range(len(conditions_arr)):
                    conditions = conditions_arr[i]
                    reward_rate = 0
                    remark = ''
                    for index, condition in conditions.iterrows():
                        if condition['is_foreign'] and condition['is_foreign'] is not isForeign:
                            continue
                        if float(condition['min_dollar_spent']) > amount:
                            continue
                        if condition['merchant'] is not None and condition['merchant'].upper() not in row["merchant"].upper():
                            continue

                        if condition['custom_category_name'] is not None:
                            custom_cat_list = customCatDf.loc[( customCatDf.apply(lambda x: x['name'].upper() in row['merchant'].upper(), axis=1) ) & (customCatDf['mcc'] == int(row['mcc'])), 'merchant'].tolist()
                            if condition['custom_category_name'] not in custom_cat_list:
                                continue

                        if reward_rate < float(condition['reward_rate']):
                            reward_rate = float(condition['reward_rate'])
#                             {condition['title']}-
                            
                            if row['cardTypeId'] == 1:
                                rate = round(condition['reward_rate']*100,2)
                            else:
                                rate = round(condition['reward_rate'],2)
                            
                            remark =  f"{remarks_mapping['default'][i]} giving {rate} {remarks_mapping['rate'][row['cardTypeId'] - 1]}/SGD" 
                            
                    if reward_rate != 0: 
#                         print(reward_rate, row['merchant'], amount)
                        amount *= reward_rate
                        row[remarks_mapping['cols'][i]] = amount
                    row['remark'] += f"{remark},"
                        
        except Exception as e:
            print(e)
            row['remark'] = f"Error in processing, {e}"
            row['rewardAmountBase'] = None
        yield Row(**row)

In [285]:
# s3_df_limit = s3_df.limit(10)

In [520]:
# Get Exclusion list
mccExclusionDf = spark.read.format("jdbc").option('driver', 'com.mysql.jdbc.Driver').option("url", campaign_db2_c.jdbc_url).option("user", campaign_db2_c.user).option("password", campaign_db2_c.password).option("dbtable", "mcc_exclusion").load()
mccExclusionDf = mccExclusionDf.filter(col("tenant") == TENANT).toPandas()

# Campaign Rules
campaignRulesDf = spark.read.format("jdbc").option('driver', 'com.mysql.jdbc.Driver').option("url", campaign_db2_c.jdbc_url).option("user", campaign_db2_c.user).option("password", campaign_db2_c.password).option("dbtable", "campaign").load()
campaignRulesDf = campaignRulesDf.filter(col("is_active")== True).toPandas()
#     baseDf = campaignRulesDf.filter(col('title') == 'base')
#     catDf = campaignRulesDf.filter(col('title') == 'category')
#     campaignDf = campaignRulesDf.filter(col('end_date').isNotNull())

# Custom Category
customCatDf = spark.read.format("jdbc").option('driver', 'com.mysql.jdbc.Driver').option("url", campaign_db2_c.jdbc_url).option("user", campaign_db2_c.user).option("password", campaign_db2_c.password).option("dbtable", "custom_category").load().toPandas()
# mcc
#     campaignDf = spark.read.format("jdbc").option('driver', 'com.mysql.jdbc.Driver').option("url", campaign_db2_c.jdbc_url).option("user", campaign_db2_c.user).option("password", campaign_db2_c.password).option("dbtable", "mcc").load()


# map(lambda row: row.asDict(), df.collect())

In [545]:
campaign_db2_c.close_conn()
s3_df_rdd = s3_df.rdd.mapPartitions( partial(processingFunc, conversionRateMapping = conversionRateMapping, mccExclusionDf=mccExclusionDf, campaignRulesDf=campaignRulesDf, customCatDf= customCatDf) )

In [546]:
s3_df_rdd.getNumPartitions()

1

In [559]:
s3_df = s3_df_rdd.toDF()

In [549]:
df.select(['merchant','mcc','currency', 'amount', 'remark', "rewardAmountBase","rewardAmountCat" , "rewardAmountCampaign", 'card_type']).limit(20).show()

+--------------------+----+--------+-------+--------------------+-----------------+---------------+--------------------+-------------+
|            merchant| mcc|currency| amount|              remark| rewardAmountBase|rewardAmountCat|rewardAmountCampaign|    card_type|
+--------------------+----+--------+-------+--------------------+-----------------+---------------+--------------------+-------------+
|               Smith|8173|     USD| 580.94|Base reward givin...|784.2690000000001|            0.0|                 0.0|scis_shopping|
|       Langworth Inc|9997|     SGD|  64.77|Base reward givin...|            64.77|            0.0|                 0.0|scis_shopping|
|     Bahringer Group|8462|     SGD|  37.72|Base reward givin...|            37.72|            0.0|                 0.0|scis_shopping|
|Transaction Recor...|   0|     SGD|1252.95|Base reward givin...|          1252.95|            0.0|              5011.8|scis_shopping|
|           Bayer LLC|1047|     SGD| 158.54|Base reward

In [558]:
df = df.toPandas()
df[['merchant','mcc','currency', 'amount', 'remark', "rewardAmountBase","rewardAmountCat" , "rewardAmountCampaign", 'card_type']]

,merchant,mcc,currency,amount,remark,rewardAmountBase,rewardAmountCat,rewardAmountCampaign,card_type
0,Smith,8173,USD,580.94,"Base reward giving 1.0 points/SGD,,,",784.269,0.0,0.0,scis_shopping
1,Langworth Inc,9997,SGD,64.77,"Base reward giving 1.0 points/SGD,,,",64.770,0.0,0.0,scis_shopping
2,Bahringer Group,8462,SGD,37.72,"Base reward giving 1.0 points/SGD,,,",37.720,0.0,0.0,scis_shopping
3,Transaction Record Grab Foods,0,SGD,1252.95,"Base reward giving 1.0 points/SGD,,Campaign re...",1252.950,0.0,5011.8,scis_shopping
4,Bayer LLC,1047,SGD,158.54,"Base reward giving 1.0 points/SGD,,,",158.540,0.0,0.0,scis_shopping


In [560]:
errorDf = errorDf.union(s3_df.filter(col('rewardAmountBase').isNull()).select(errorDf.columns) )
s3_df = s3_df.filter(col('rewardAmountBase').isNotNull())

print(errorDf.count())
print(s3_df.count())

0
5


In [422]:
errorDf.coalesce(1).write.format('csv').option("header", "true").save("./error.csv") #s3://filepath

In [569]:
def sendToQueue(partitionData, queueName, tenant):
    sqs = boto3.resource('sqs')
    queue = sqs.get_queue_by_name(QueueName=queueName)
    
    for row in partitionData:
        row = row.asDict()
        remarks = row['remark'].split(',')
        amount_arr = [row['rewardAmountBase'],row['rewardAmountCat'],row['rewardAmountCampaign'] ]
        for i in range(3):
            if remarks[i] == '':
                continue

            message_body = {
                "tenant": tenant.upper(),
                "transactionId": row['transaction_id'],
                "transactionDate": row['transaction_date'],
                "cardId": row['card_id'],
                "merchant": row['merchant'],
                "mcc": row['mcc'],
                "currency": row['currency'],
                "amount": float(row['amount']),
                "rewardAmount": round(float(amount_arr[i]),2),
                "remarks": remarks[i]
            }
            print(message_body)
            try:
                response = queue.send_message(MessageBody=message_body)
            except ClientError as error:
                sqs = boto3.resource('sqs')
                queue = sqs.get_queue_by_name(QueueName=queueName)
                response = queue.send_message(MessageBody=message_body)
            


In [570]:
s3_df.foreachPartition( partial(sendToQueue, queueName=queueName, tenant = TENANT) )

In [571]:
import requests

In [ ]:
{
    "tenant":"SCIS",
    "transactionId":"a750165d557443feb043693cac84515878de0f394413be869f0b62c21f119705",
    "transactionDate":"2021-09-03",
    "cardId":"605ec742-0ac4-43e7-8e3b-8bc63379a2fe",
    "merchant":"Transaction Record Grab Foods",
    "mcc":0,
    "currency":"SGD",
    "amount":1252.95,
    "rewardAmount":5011.8,
    "remarks":"Campaign reward 4.0"
}

In [477]:
test_sample = [ {
   "id": "055cd9e8-e9f4-4f8d-9190-1d16251727f7",   "tenant": "scis",
   "transaction_id": "5cbd48a13727468feab3446eaefe676f9851dff5632a26f6fe00ef9eb44cf1c5",   "merchant": "Bayer LLC",
   "mcc": 1047,   "currency": "SGD",
   "amount": 158.54,   "transaction_date": "3/9/2021",
   "card_id": "605ec742-0ac4-43e7-8e3b-8bc63379a2fe",   "cardType": "scis_shopping"
 }, {
   "id": "4a25d76c-2db7-47b2-8618-579ef6f203d6",   "tenant": "scis",
   "transaction_id": "a750165d557443feb043693cac84515878de0f394413be869f0b62c21f119705",   "merchant": "Transaction Record Grab Foods",
   "mcc": 0,   "currency": "SGD",
   "amount": 1252.95,   "transaction_date": "3/9/2021",
   "card_id": "605ec742-0ac4-43e7-8e3b-8bc63379a2fe",   "cardType": "scis_shopping"
 },
 {   "id": "43dd9d38-9b7f-4a28-a131-b8bb29d5093b",
   "tenant": "scis",   "transaction_id": "41a928d858e21fd360e2f33203381868ffa2285b0f5005e2a3c5ad08b4d9895e",
   "merchant": "Bahringer Group",   "mcc": 8462,
   "currency": "SGD",   "amount": 37.72,
   "transaction_date": "3/9/2021",   "card_id": "2379df1a-d1fe-4150-bfb4-b738543bd107",
   "cardType": "scis_shopping" },
 {   "id": "5b2d5c02-75fc-404c-bc85-15b88ee8eb2a",
   "tenant": "scis",   "transaction_id": "8a46f127d7e3ec1e56489937a2d4a2a44d1852f10403f3186ccc184b39299d2f",
   "merchant": "Langworth Inc",   "mcc": 9997,
   "currency": "SGD",   "amount": 64.77,
   "transaction_date": "3/9/2021",   "card_id": "2379df1a-d1fe-4150-bfb4-b738543bd107",
   "cardType": "scis_shopping" },
 {   "id": "20285a85-788f-47e6-8669-15f75e8fe6ff",
   "tenant": "scis",   "transaction_id": "3cc0c41872cc33b39c8d249841cccfaeb8fba2b672020d9041f03a939205f77b",
   "merchant": "Smith",   "mcc": 8173,
   "currency": "USD",   "amount": 580.94,
   "transaction_date": "3/9/2021",   "card_id": "2379df1a-d1fe-4150-bfb4-b738543bd107",
   "cardType": "scis_shopping" }
]

In [481]:
import pandas as pd
df = pd.DataFrame(test_sample)

In [483]:
df.to_csv("testSpend.csv")